In [ ]:
import os
import time
import shutil
import warnings
import time

# Suppress future warnings
warnings.filterwarnings('ignore', category=FutureWarning)

# Create constraints file
!echo "numpy<2.0" > /kaggle/working/constraints.txt

# Set environment variable for the entire notebook
import os
os.environ['PIP_CONSTRAINT'] = '/kaggle/working/constraints.txt'

# --------------------------
# 0. 清理旧进程
# --------------------------
print("清理旧进程...")
!pkill -f 'python main.py' || true
!pkill -f 'pinggy' || true

# --------------------------
# 0.5 检测和配置多GPU
# --------------------------
print("\n" + "="*60)
print("检测 GPU 配置...")
print("="*60)

try:
    import torch
    num_gpus = torch.cuda.device_count()
    print(f"✓ 检测到 {num_gpus} 个 GPU")
    
    for i in range(num_gpus):
        print(f"  GPU {i}: {torch.cuda.get_device_name(i)}")
        props = torch.cuda.get_device_properties(i)
        print(f"    - 显存: {props.total_memory / 1024**3:.2f} GB")
    
    # 设置环境变量以启用多GPU
    if num_gpus > 1:
        print(f"\n✓ 配置多GPU模式 ({num_gpus} GPUs)")
        os.environ['CUDA_VISIBLE_DEVICES'] = ','.join(map(str, range(num_gpus)))
    else:
        print("\n⚠ 仅检测到1个GPU")
        
except Exception as e:
    print(f"✗ GPU检测失败: {e}")

print("="*60)

# --------------------------
# 1. 系统依赖
# --------------------------
print("安装系统库...")
!apt-get update -qq
!apt-get install -y -qq libgl1 libglib2.0-0 ffmpeg

# --------------------------
# 2. 克隆 ComfyUI
# --------------------------
print("克隆 ComfyUI 仓库...")
if not os.path.exists("ComfyUI"):
    !git clone https://github.com/comfyanonymous/ComfyUI.git

%cd ComfyUI
print(f"当前工作目录: {os.getcwd()}")
print("更新 ComfyUI...")
!git pull

# --------------------------
# 3. 安装 Python 库 (优化版本组合)
# --------------------------
print("安装额外依赖...")
!pip install -q mediapipe
!pip uninstall onnxruntime -y
!pip install onnxruntime-gpu

import onnxruntime as ort
print(ort.get_available_providers())
# Should show: ['CUDAExecutionProvider', 'CPUExecutionProvider']

# SAM-2 needs to be installed from git
!pip install -q git+https://github.com/facebookresearch/segment-anything-2.git

print("卸载不兼容库...")
!pip uninstall -y torch torchvision torchaudio xformers

print("安装最新 PyTorch nightly (WanVideoWrapper 要求 2.7.0.dev20250226+)...")
!pip install --pre --upgrade --extra-index-url https://download.pytorch.org/whl/nightly/cu121 torch torchvision torchaudio

print("安装兼容的 xformers...")
!pip install -q xformers

print("安装 ComfyUI 其余依赖...")

# Downgrade numpy to avoid incompatibility with custom nodes

!pip install -q -r requirements.txt --no-deps
!pip install -q torchsde einops transformers safetensors aiohttp accelerate pyyaml opencv-python pillow scipy imageio[ffmpeg] moviepy huggingface_hub gguf
!pip install -q pydantic-settings python-dotenv

# --------------------------
# 4. 验证安装
# --------------------------
print("\n" + "="*50)
print("验证 PyTorch 和 xformers...")
print("="*50)
try:
    import torch
    import torchvision
    import xformers
    import xformers.ops
    
    print(f"✓ PyTorch: {torch.__version__}")
    print(f"✓ TorchVision: {torchvision.__version__}")
    print(f"✓ xformers: {xformers.__version__}")
    print(f"✓ CUDA 可用: {torch.cuda.is_available()}")
    
    if torch.cuda.is_available():
        num_gpus = torch.cuda.device_count()
        print(f"✓ GPU 数量: {num_gpus}")
        for i in range(num_gpus):
            print(f"  - GPU {i}: {torch.cuda.get_device_name(i)}")
        print(f"✓ CUDA 版本: {torch.version.cuda}")
    
    # 验证 RMSNorm 可用性
    if hasattr(torch.nn, 'RMSNorm'):
        print("✓ RMSNorm 可用 (WanVideoWrapper 将正常工作)")
    else:
        print("✗ RMSNorm 不可用")
        
    print("="*50)
except Exception as e:
    print(f"✗ 验证失败: {e}")
    print("="*50)

# --------------------------
# 5. 使用 Kaggle Input 的模型 (通过符号链接)
# --------------------------
kaggle_input_dir = "/kaggle/input"

print("\n使用本地 Kaggle Input 模型目录:", kaggle_input_dir)

# 创建模型目录
model_dirs = {
    "diffusion_models": "./models/diffusion_models",
    "text_encoders": "./models/text_encoders",
    "vae": "./models/vae",
    "loras": "./models/loras",
    "clip_vision":"./models/clip_vision"
}

for dir_path in model_dirs.values():
    os.makedirs(dir_path, exist_ok=True)

# 使用符号链接代替复制文件
print("\n创建模型文件符号链接 (节省存储空间)...")
print("="*60)

link_map = {
    "wan-animate-gguf-text-encoder": "./models/text_encoders",
    "wan-animate-vae": "./models/vae", 
    "wan-animate-q3-k-m-gguf": "./models/unet",
    "wan-animate-loras": "./models/loras",
    "wan-animate-clip-vision":"./models/clip_vision"
}

for src_name, dst_path in link_map.items():
    src_path = f"{kaggle_input_dir}/{src_name}"
    if os.path.exists(src_path):
        print(f"\n处理: {src_name}")
        try:
            # 列出源目录中的文件
            files = os.listdir(src_path)
            if not files:
                print(f"  ⚠ 空目录: {src_name}")
                continue
            
            # 为每个文件创建符号链接
            for file in files:
                src_file = os.path.join(src_path, file)
                dst_file = os.path.join(dst_path, file)
                
                # 如果目标已存在，先删除
                if os.path.exists(dst_file) or os.path.islink(dst_file):
                    os.remove(dst_file)
                    print(f"  - 删除旧链接: {file}")
                
                # 创建符号链接
                os.symlink(src_file, dst_file)
                print(f"  ✓ 链接: {file}")
                
        except Exception as e:
            print(f"  ✗ 错误: {e}")
    else:
        print(f"  ⚠ 未找到: {src_name}")

print("\n" + "="*60)
print("模型文件链接列表:")
print("-" * 50)
for name, path in model_dirs.items():
    print(f"\n{name}:")
    !ls -lh {path}/ 2>/dev/null || echo "  (空)"

# 验证符号链接
print("\n验证符号链接...")
for name, path in model_dirs.items():
    if os.path.exists(path):
        files = os.listdir(path)
        for file in files:
            full_path = os.path.join(path, file)
            if os.path.islink(full_path):
                target = os.readlink(full_path)
                print(f"  ✓ {file} -> {target}")

# --------------------------
# 6. 安装所有必需插件
# --------------------------
print("\n" + "="*60)
print("安装 ComfyUI 插件...")
print("="*60)

os.makedirs("custom_nodes", exist_ok=True)
%cd custom_nodes

# Complete list of plugins including Kijai workflow requirements
all_plugins = {
    # Core plugins
    "ComfyUI-Manager": "https://github.com/ltdrdata/ComfyUI-Manager.git",
    "ComfyUI-GGUF": "https://github.com/city96/ComfyUI-GGUF.git",
    "ComfyUI-WanVideoWrapper": "https://github.com/kijai/ComfyUI-WanVideoWrapper.git",
    
    # Kijai workflow requirements
    "ComfyUI-KJNodes": "https://github.com/kijai/ComfyUI-KJNodes.git",
    "comfyui_controlnet_aux": "https://github.com/Fannovel16/comfyui_controlnet_aux.git",
    "ComfyUI-VideoHelperSuite": "https://github.com/Kosinkadink/ComfyUI-VideoHelperSuite.git",
    "ComfyUI-segment-anything-2": "https://github.com/kijai/ComfyUI-segment-anything-2.git",
    "ComfyUI-Advanced-ControlNet": "https://github.com/Kosinkadink/ComfyUI-Advanced-ControlNet.git",
    "comfyui-tooling-nodes": "https://github.com/Acly/comfyui-tooling-nodes.git",
    "ComfyUI_essentials": "https://github.com/cubiq/ComfyUI_essentials.git",
    
    # Additional required nodes
    "cg-use-everywhere": "https://github.com/chrisgoringe/cg-use-everywhere.git",
    "was-node-suite-comfyui": "https://github.com/WASasquatch/was-node-suite-comfyui.git",
    "ComfyUI-Custom-Scripts": "https://github.com/pythongosssss/ComfyUI-Custom-Scripts.git",
    "ComfyUI-Frame-Interpolation": "https://github.com/Fannovel16/ComfyUI-Frame-Interpolation.git",
    "rgthree-comfy": "https://github.com/rgthree/rgthree-comfy.git",
    "ComfyMath": "https://github.com/evanspearman/ComfyMath.git",
    "ComfyUI-Easy-Use": "https://github.com/yolain/ComfyUI-Easy-Use.git",
    "comfyui-mixlab-nodes": "https://github.com/MixLabPro/comfyui-mixlab-nodes.git"
}

for plugin_name, plugin_url in all_plugins.items():
    print(f"\n{'='*60}")
    print(f"处理: {plugin_name}")
    print(f"{'='*60}")
    
    if not os.path.exists(plugin_name):
        print(f"  → 克隆 {plugin_name}...")
        !git clone {plugin_url}
        
        # Install requirements
        req_file = f"{plugin_name}/requirements.txt"
        if os.path.exists(req_file):
            print(f"  → 安装依赖...")
            !pip install -q -r {req_file}
            
        print(f"  ✓ {plugin_name} 完成")
    else:
        print(f"  → {plugin_name} 已存在，更新中...")
        %cd {plugin_name}
        !git pull -q
        
        # Update requirements
        if os.path.exists("requirements.txt"):
            !pip install -q -r requirements.txt
        %cd ..
        print(f"  ✓ {plugin_name} 更新完成")

%cd ..
print("\n" + "="*60)
print(f"✓ 所有插件安装完成！")
print(f"当前工作目录: {os.getcwd()}")
print("="*60)

# --------------------------
# 6.5 配置 ComfyUI 多GPU支持
# --------------------------
print("\n" + "="*60)
print("配置 ComfyUI 多GPU支持...")
print("="*60)

# Remove or fix the extra_model_paths.yaml file to prevent startup errors
config_file = "extra_model_paths.yaml"
if os.path.exists(config_file):
    os.remove(config_file)
    print(f"✓ 已删除旧配置文件: {config_file}")

# ComfyUI will use default paths, which works better with multi-GPU
print("✓ 使用默认配置 (自动多GPU支持)")
print("="*60)

# Suppress warnings in environment
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['PYTHONWARNINGS'] = 'ignore::FutureWarning'

# Change to ComfyUI directory
%cd /kaggle/working/ComfyUI

print("\n🚀 启动 ComfyUI 服务器 (多GPU模式)...")

# Check GPU count one more time
try:
    import torch
    num_gpus = torch.cuda.device_count()
    if num_gpus > 1:
        print(f"✓ 启用 {num_gpus} GPU 模式")
        print(f"  CUDA_VISIBLE_DEVICES: {os.environ.get('CUDA_VISIBLE_DEVICES', 'all')}")
    print()
except:
    pass

# --------------------------
# 7. 启动 ComfyUI 作为后台进程
# --------------------------
import subprocess
import time
import requests
import sys

print("\n" + "="*60)
print("🚀 启动 ComfyUI 服务器 (后台进程模式)...")
print("="*60)

# Ensure we're in the ComfyUI directory
os.chdir('/kaggle/working/ComfyUI')
print(f"当前目录: {os.getcwd()}")

# Configure launch arguments
comfy_args = [
    sys.executable,  # Use current Python interpreter
    'main.py',
    '--listen', '0.0.0.0',
    '--port', '8188',
    '--preview-method', 'auto'
]

# Add multi-GPU support if available
try:
    import torch
    num_gpus = torch.cuda.device_count()
    if num_gpus > 1:
        print(f"✓ 配置多GPU模式 ({num_gpus} GPUs)")
        # ComfyUI will automatically use all available GPUs
except:
    pass

# Start ComfyUI as background subprocess
print(f"\n启动命令: {' '.join(comfy_args)}")
print("="*60)

comfy_process = subprocess.Popen(
    comfy_args,
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    cwd='/kaggle/working/ComfyUI',
    bufsize=1,
    universal_newlines=True
)

print(f"✓ ComfyUI 进程已启动")
print(f"  PID: {comfy_process.pid}")
print(f"  状态: {'运行中' if comfy_process.poll() is None else '已停止'}")

# --------------------------
# 8. 等待 ComfyUI 就绪
# --------------------------
def wait_for_comfyui(max_attempts=60, delay=2):
    """等待 ComfyUI 服务器就绪"""
    print(f"\n⏳ 等待 ComfyUI 启动 (最多 {max_attempts * delay} 秒)...")
    print("-" * 60)
    
    for i in range(max_attempts):
        # Check if process is still running
        if comfy_process.poll() is not None:
            print(f"\n✗ ComfyUI 进程意外终止!")
            print("\n最后的输出:")
            if comfy_process.stdout:
                print(comfy_process.stdout.read())
            if comfy_process.stderr:
                print("\n错误输出:")
                print(comfy_process.stderr.read())
            return False
        
        try:
            # Try to connect to ComfyUI API
            response = requests.get(
                "http://127.0.0.1:8188/system_stats",
                timeout=2
            )
            
            if response.status_code == 200:
                print(f"\n✓ ComfyUI 已就绪! (耗时 {(i+1) * delay} 秒)")
                print("-" * 60)
                
                # Print system stats
                try:
                    stats = response.json()
                    print("\n系统状态:")
                    if 'system' in stats:
                        sys_info = stats['system']
                        print(f"  • OS: {sys_info.get('os', 'N/A')}")
                        print(f"  • Python: {sys_info.get('python_version', 'N/A')}")
                    
                    if 'devices' in stats:
                        devices = stats['devices']
                        print(f"\n  • GPU 设备:")
                        for i, device in enumerate(devices):
                            print(f"    - GPU {i}: {device.get('name', 'N/A')}")
                            vram_total = device.get('vram_total', 0) / (1024**3)
                            print(f"      VRAM: {vram_total:.2f} GB")
                except:
                    pass
                
                print("="*60)
                return True
                
        except requests.exceptions.RequestException:
            pass
        except Exception as e:
            pass
        
        # Progress indicator
        time.sleep(delay)
        if (i + 1) % 5 == 0:
            print(f"  尝试 {i+1}/{max_attempts} - 仍在启动中...")
    
    print(f"\n⚠️ 超时: ComfyUI 未能在 {max_attempts * delay} 秒内启动")
    print("\n检查进程状态...")
    print(f"  进程状态: {'运行中' if comfy_process.poll() is None else '已停止'}")
    
    return False

# Wait for ComfyUI to be ready
if wait_for_comfyui():
    print("\n🎉 ComfyUI 服务器运行正常!")
    print(f"    → 本地访问: http://127.0.0.1:8188")
    print(f"    → 进程 PID: {comfy_process.pid}")
    print("\n✓ 可以继续运行下一个单元格启动 Gradio")
else:
    print("\n⚠️  ComfyUI 启动可能遇到问题")
    print("    → 建议检查下面的日志输出")
    print("    → 或运行调试单元格查看详细错误")

print("="*60)

# --------------------------
# 9. 可选: 实时查看 ComfyUI 日志
# --------------------------
# 如果需要查看实时日志，可以运行这个单元格:

import threading

def stream_output(pipe, label):
    '''实时打印进程输出'''
    for line in iter(pipe.readline, ''):
        if line:
            print(f"[{label}] {line.rstrip()}")

# 创建日志线程
stdout_thread = threading.Thread(
    target=stream_output, 
    args=(comfy_process.stdout, "STDOUT"),
    daemon=True
)
stderr_thread = threading.Thread(
    target=stream_output,
    args=(comfy_process.stderr, "STDERR"),
    daemon=True
)

stdout_thread.start()
stderr_thread.start()

print("✓ 日志监控已启动 (后台运行)")

# --------------------------
# 10. 调试单元格 (可选)
# --------------------------
# 如果 ComfyUI 启动失败，运行这个来查看详细信息:

print("="*60)
print("ComfyUI 进程诊断")
print("="*60)

print(f"\n进程 PID: {comfy_process.pid}")
print(f"进程状态: {comfy_process.poll()}")
print(f"运行中: {comfy_process.poll() is None}")

if comfy_process.poll() is not None:
    print("\n⚠️  进程已停止")
    
    print("\n标准输出:")
    print("-" * 60)
    if comfy_process.stdout:
        stdout = comfy_process.stdout.read()
        print(stdout if stdout else "(无输出)")
    
    print("\n标准错误:")
    print("-" * 60)
    if comfy_process.stderr:
        stderr = comfy_process.stderr.read()
        print(stderr if stderr else "(无错误)")
else:
    print("\n✓ 进程仍在运行")
    
    # Test API connection
    try:
        response = requests.get("http://127.0.0.1:8188/system_stats", timeout=2)
        print(f"\n✓ API 响应: {response.status_code}")
        print(response.json())
    except Exception as e:
        print(f"\n✗ API 连接失败: {e}")

print("="*60)

In [ ]:
# GRADIO + COMFYUI API (In case if Localtunnel connection is unstable)

!pip install gradio requests websocket-client pillow

import json
import requests
import gradio as gr
import websocket
import uuid
import urllib.request
import urllib.parse
from PIL import Image
import io
import os
import time

class ComfyUIAPI:
    def __init__(self, server_address="127.0.0.1:8188"):
        self.server_address = server_address
        self.client_id = str(uuid.uuid4())
        
    def queue_prompt(self, prompt):
        """Queue a prompt to ComfyUI"""
        p = {"prompt": prompt, "client_id": self.client_id}
        data = json.dumps(p).encode('utf-8')
        req = urllib.request.Request(f"http://{self.server_address}/prompt", data=data)
        return json.loads(urllib.request.urlopen(req).read())
    
    def upload_image(self, image_path, subfolder="", overwrite=False):
        """Upload an image to ComfyUI"""
        with open(image_path, 'rb') as f:
            files = {'image': f}
            data = {
                'subfolder': subfolder,
                'overwrite': str(overwrite).lower()
            }
            response = requests.post(
                f"http://{self.server_address}/upload/image",
                files=files,
                data=data
            )
        return response.json()
    
    def upload_video(self, video_path, subfolder="", overwrite=True):
        """Upload a video to ComfyUI"""
        with open(video_path, 'rb') as f:
            files = {'image': (os.path.basename(video_path), f, 'video/mp4')}
            data = {
                'subfolder': subfolder,
                'overwrite': str(overwrite).lower(),
                'type': 'input'
            }
            response = requests.post(
                f"http://{self.server_address}/upload/image",
                files=files,
                data=data
            )
        return response.json()
    
    def get_history(self, prompt_id):
        """Get the execution history for a prompt"""
        req = urllib.request.Request(f"http://{self.server_address}/history/{prompt_id}")
        return json.loads(urllib.request.urlopen(req).read())
    
    def get_image(self, filename, subfolder, folder_type):
        """Download an image from ComfyUI"""
        data = {"filename": filename, "subfolder": subfolder, "type": folder_type}
        url_values = urllib.parse.urlencode(data)
        url = f"http://{self.server_address}/view?{url_values}"
        with urllib.request.urlopen(url) as response:
            return response.read()
    
    def track_progress(self, prompt_id, progress_callback=None):
        """Track workflow execution progress via WebSocket"""
        ws = websocket.WebSocket()
        ws.connect(f"ws://{self.server_address}/ws?clientId={self.client_id}")
        
        while True:
            out = ws.recv()
            if isinstance(out, str):
                message = json.loads(out)
                
                if message['type'] == 'progress' and progress_callback:
                    data = message['data']
                    progress_callback(data.get('value', 0), data.get('max', 100))
                
                if message['type'] == 'executing':
                    data = message['data']
                    if data['node'] is None and data['prompt_id'] == prompt_id:
                        break
        ws.close()

class WanVideoGradioApp:
    def __init__(self):
        self.api = ComfyUIAPI()
        self.workflow = None
        
    def load_workflow(self, workflow_file):
        """Load the WanVideo workflow JSON file"""
        try:
            if workflow_file is None:
                return "⚠ Please upload a workflow file"
                
            with open(workflow_file.name, 'r') as f:
                self.workflow = json.load(f)
            return f"✓ Workflow loaded successfully! Found {len(self.workflow)} nodes.\nReady to process videos."
        except Exception as e:
            return f"✗ Error loading workflow: {str(e)}"
    
    def run_workflow(self, prompt_text, negative_prompt, input_image, input_video, 
                    length_seconds, framerate, resolution, interpolation_factor, seed):
        """Execute the WanVideo workflow with provided inputs"""
        if not self.workflow:
            return "⚠ Please load a workflow first!", None, None
        
        try:
            status_updates = []
            
            # Upload image
            if input_image:
                status_updates.append("📤 Uploading image...")
                img_result = self.api.upload_image(input_image)
                self.workflow["64"]["inputs"]["image"] = img_result['name']
                status_updates.append(f"✓ Image uploaded: {img_result['name']}")
            
            # Upload video
            if input_video:
                status_updates.append("📤 Uploading video...")
                vid_result = self.api.upload_video(input_video)
                self.workflow["13"]["inputs"]["video"] = vid_result['name']
                status_updates.append(f"✓ Video uploaded: {vid_result['name']}")
            
            # Update text prompts
            if prompt_text:
                self.workflow["124"]["inputs"]["text"] = prompt_text
            if negative_prompt:
                self.workflow["122"]["inputs"]["text"] = negative_prompt
            
            # Update parameters
            self.workflow["28"]["inputs"]["value"] = float(length_seconds)
            self.workflow["30"]["inputs"]["value"] = float(framerate)
            self.workflow["32"]["inputs"]["value"] = int(resolution)
            self.workflow["29"]["inputs"]["value"] = int(interpolation_factor)
            self.workflow["90"]["inputs"]["seed"] = int(seed)
            
            status_updates.append("⚙️ Starting workflow execution...")
            
            # Queue the prompt
            result = self.api.queue_prompt(self.workflow)
            prompt_id = result['prompt_id']
            status_updates.append(f"✓ Workflow queued (ID: {prompt_id[:8]}...)")
            
            # Track progress
            status_updates.append("🔄 Processing... This may take several minutes.")
            self.api.track_progress(prompt_id)
            
            status_updates.append("✓ Workflow completed!")
            status_updates.append("📦 Retrieving outputs...")
            
            # Get output
            time.sleep(2)  # Wait for files to be written
            history = self.api.get_history(prompt_id)[prompt_id]
            
            # Get output images/videos
            output_images = []
            output_info = []
            
            for node_id in history['outputs']:
                node_output = history['outputs'][node_id]
                
                # Get images
                if 'images' in node_output:
                    for image in node_output['images']:
                        try:
                            image_data = self.api.get_image(
                                image['filename'], 
                                image['subfolder'], 
                                image['type']
                            )
                            img = Image.open(io.BytesIO(image_data))
                            output_images.append(img)
                            output_info.append(f"Image: {image['filename']}")
                        except:
                            pass
                
                # Get videos
                if 'gifs' in node_output:
                    for video in node_output['gifs']:
                        output_info.append(f"Video: {video['filename']}")
            
            status_updates.append(f"✓ Found {len(output_info)} outputs")
            
            final_status = "\n".join(status_updates)
            
            if output_images:
                return final_status, output_images[0], "\n".join(output_info)
            else:
                return final_status + "\n\n✓ Video generation complete! Check ComfyUI output folder.", None, "\n".join(output_info)
                
        except Exception as e:
            error_msg = f"✗ Error executing workflow: {str(e)}"
            import traceback
            error_msg += f"\n\nDetails:\n{traceback.format_exc()}"
            return error_msg, None, None

# Initialize the app
app = WanVideoGradioApp()

# Create Gradio interface
with gr.Blocks(title="WanVideo ComfyUI Controller", theme=gr.themes.Soft()) as demo:
    gr.Markdown("""
    # 🎬 WanVideo Animation Controller
    ### Control WanVideo workflow via ComfyUI API with Gradio
    """)
    
    with gr.Row():
        workflow_file = gr.File(
            label="📁 Upload Workflow JSON (wan_2_2_animate.json)",
            file_types=[".json"]
        )
        load_btn = gr.Button("Load Workflow", variant="primary", size="sm")
    
    workflow_status = gr.Textbox(
        label="Workflow Status",
        interactive=False,
        lines=2
    )
    
    gr.Markdown("---")
    gr.Markdown("### 🎨 Input Configuration")
    
    with gr.Row():
        with gr.Column():
            gr.Markdown("**📝 Text Inputs**")
            prompt_text = gr.Textbox(
                label="Positive Prompt",
                placeholder="Describe the animation you want...",
                lines=3,
                value="Shrek is dancing."
            )
            negative_prompt = gr.Textbox(
                label="Negative Prompt",
                placeholder="What to avoid in the animation...",
                lines=3,
                value="overexposed, static, blurry details, low quality"
            )
        
        with gr.Column():
            gr.Markdown("**🖼️ Media Inputs**")
            input_image = gr.Image(
                label="Reference Image",
                type="filepath",
                height=200
            )
            input_video = gr.Video(
                label="Input Video (Optional)",
                height=200
            )
    
    gr.Markdown("### ⚙️ Generation Parameters")
    
    with gr.Row():
        length_seconds = gr.Slider(
            minimum=1,
            maximum=30,
            value=7,
            step=1,
            label="🎬 Length (Seconds)"
        )
        framerate = gr.Slider(
            minimum=8,
            maximum=60,
            value=16,
            step=1,
            label="🎦 Framerate (FPS)"
        )
    
    with gr.Row():
        resolution = gr.Slider(
            minimum=256,
            maximum=1024,
            value=480,
            step=64,
            label="🔲 Resolution (Width)"
        )
        interpolation_factor = gr.Slider(
            minimum=1,
            maximum=4,
            value=2,
            step=1,
            label="🔀 Interpolation Factor"
        )
    
    seed = gr.Number(
        label="🎲 Seed",
        value=777,
        precision=0
    )
    
    run_btn = gr.Button("▶ Generate Animation", variant="primary", size="lg")
    
    gr.Markdown("---")
    gr.Markdown("### 📊 Results")
    
    with gr.Row():
        with gr.Column():
            execution_status = gr.Textbox(
                label="Execution Log",
                interactive=False,
                lines=10
            )
        with gr.Column():
            output_image = gr.Image(
                label="Preview Frame"
            )
    
    output_files = gr.Textbox(
        label="Output Files",
        interactive=False,
        lines=3
    )
    
    gr.Markdown("""
    ---
    ### 📖 Instructions:
    
    1. **Load Workflow**: Upload the `wan_2_2_animate.json` file
    2. **Set Prompts**: Describe the animation you want to create
    3. **Upload Media**: 
       - **Required**: Reference image (character/subject)
       - **Optional**: Input video for motion reference
    4. **Adjust Parameters**: 
       - Length: Duration in seconds
       - Framerate: Higher = smoother but slower to generate
       - Resolution: Output width (height auto-calculated)
       - Interpolation: Multiplier for frame interpolation (2 = double frames)
    5. **Generate**: Click "Generate Animation" and wait
    
    **Output Location**: Check your ComfyUI `output` folder for the final videos:
    - `Wanimate_*.mp4` - Base animation
    - `Wanimate_Interpolated_*.mp4` - Interpolated high-FPS version
    
    **Note**: Make sure ComfyUI is running on `127.0.0.1:8188` with all required models loaded.
    """)
    
    # Event handlers
    load_btn.click(
        fn=app.load_workflow,
        inputs=[workflow_file],
        outputs=[workflow_status]
    )
    
    run_btn.click(
        fn=app.run_workflow,
        inputs=[
            prompt_text,
            negative_prompt,
            input_image,
            input_video,
            length_seconds,
            framerate,
            resolution,
            interpolation_factor,
            seed
        ],
        outputs=[execution_status, output_image, output_files]
    )
    
    # Load example
    gr.Examples(
        examples=[
            ["A person dancing energetically", "static, blurry, low quality", 5, 16, 480, 2, 777],
            ["Character walking forward confidently", "overexposed, static, ugly", 7, 16, 512, 2, 123],
        ],
        inputs=[prompt_text, negative_prompt, length_seconds, framerate, resolution, interpolation_factor, seed],
    )


demo.launch(share=True)